In [ ]:
import sys 
sys.path.append("..")
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error
import os
import time
from multiprocessing import Pool
import torch
from torch_geometric.data import Data, DataLoader
import random
#import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.nn import MessagePassing, GraphNorm
from torch_geometric.nn.aggr import AttentionalAggregation
from torch_scatter import scatter
import pandas as pd
from torch_scatter import scatter_add
import os.path as osp
from torch_geometric.nn import global_add_pool
from torch_geometric.utils import softmax
import math
from torch_geometric.loader import DataLoader
from Graph_ import Mol2Graph
from cal_descriptors_ import cal_des
import fit
from typing import Union, Optional
from torch_geometric.typing import (Adj, Size, OptTensor, PairTensor)  #
#import torch
from torch import Tensor
#import torch.nn.functional as F
from torch.nn import Parameter, Linear, BatchNorm1d
from torch_sparse import SparseTensor
#from torch_geometric.nn.conv import MessagePassing
#from torch_geometric.utils import softmax
from torch_geometric.nn.inits import glorot  #


In [ ]:
from load_data import load_dataset
train_data,test_data = load_dataset('data/emi','emi')

# data pipeline

In [ ]:
path = 'data/emi/data.txt'
items = [i.strip().split('\t') for i in open(path).readlines()] 
number = len(items)
Y = np.array([float(i[-1]) for i in items])
std = Y.std()
mean = Y.mean()   
print(mean,std)

# GCN_conv

In [ ]:
from torch_geometric.nn import GCNConv
 
class GraphConvolution(torch.nn.Module):
    def __init__(self,input_size, output_size):
        super(GraphConvolution, self).__init__()
        self.conv1 = GCNConv(input_size,output_size)
        self.relu = torch.nn.ReLU()
 
 
    def forward(self, x, edge_index, edge_weight):
        '''
        GCN
        '''
        x = self.relu(self.conv1(x, edge_index,edge_weight))
        #x = self.MLP(x)
 
        return x

# Conv

In [ ]:
class OledConv(torch.nn.Module):  #
    def __init__(self, layer=5,hidden_size=128):  
        super().__init__()  # "Add" aggregation (Step 5)
        self.encoder = nn.Linear(46,hidden_size)
        self.gcn = nn.ModuleList([GraphConvolution(hidden_size,hidden_size) for i in range(layer)])
        self.norms = nn.ModuleList([nn.BatchNorm1d(hidden_size) for _ in range(layer)])
                
        self.encoder_sol = nn.Linear(46,hidden_size)
        self.gcn_sol = nn.ModuleList([GraphConvolution(hidden_size,hidden_size) for i in range(layer)])
        self.norms_sol = nn.ModuleList([nn.BatchNorm1d(hidden_size) for _ in range(layer)])
        
        self.gru = nn.ModuleList([nn.GRU(hidden_size,hidden_size) for _ in range(layer)])
        self.gru_sol = nn.ModuleList([nn.GRU(hidden_size,hidden_size) for _ in range(layer)])
        
        self.MLP = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size * 2),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(hidden_size * 2, hidden_size *4)
            )
        self.MLP_sol = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size *2),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(hidden_size * 2, hidden_size *4)
            )
        self.MLP_emb = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * 8, hidden_size * 4),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(hidden_size * 4, hidden_size *2)
            )
        self.MLP_pre = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * 2, hidden_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(hidden_size, 1)
            )
        
        self.out_dim = hidden_size
        
    def forward(self, data,data_sol):   #
        
        #for chromophere  
        x = self.encoder(data.x)
        x_sol = self.encoder(data_sol.x)

        h = x.unsqueeze(0)
        h_ = x_sol.unsqueeze(0)
        for i,(gcn,gcn_sol,norm,norm_sol,gru,gru_sol) in enumerate(zip(self.gcn,self.gcn_sol,self.norms,self.norms_sol,self.gru,self.gru_sol)):
            x = gcn(x,data.edge_index,data.edge_weight)    
            x,h = gru(x.unsqueeze(0),h)
            x = norm(x.squeeze(0))
            x_sol = gcn_sol(x_sol,data_sol.edge_index,data_sol.edge_weight)
            x_sol,h_ = gru_sol(x_sol.unsqueeze(0),h_)
            x_sol = norm_sol(x_sol.squeeze(0))
            
        x = scatter_add(x.reshape(-1,self.out_dim),dim=0,index=data.batch)
        x_sol = scatter_add(x_sol.reshape(-1,self.out_dim),dim=0,index=data_sol.batch)

        x = self.MLP(x)
        x_sol = self.MLP_sol(x_sol)
        
        x = torch.cat([x,x_sol],dim=1)
        x = self.MLP_emb(x)
        
        out = self.MLP_pre(x)
        
        return out.view(-1)

In [ ]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=1,)
valid_loader = DataLoader(test_data, batch_size=128, shuffle=0,)

In [ ]:
import sys 
sys.path.append("..") 
import fit
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_name = 'OLED_GNN'
data_loaders = {'train':train_loader, 'valid':valid_loader}#, 'test':test_loader}
fit.training(OledConv, data_loaders, patience = 10,n_epoch=100, snapshot_path='./snapshot/{}//'.format('GCN_emi.12.27_new'), mean=mean, std=std,)